<a href="https://colab.research.google.com/github/robitussin/CCMACLRL_EXERCISES/blob/main/Exercise7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 7: Hate Speech Classification using Multinomial Naive Bayes

Instructions:
- You do not need to split your data. Use the training, validation and test sets provided below.
- Use Multinomial Naive Bayes to train a model that can classify if a sentence is a hate speech or non-hate speech
- A sentence with a label of zero (0) is classified as non-hate speech
- A sentence with a label of one (1) is classified as a hate speech

Apply text pre-processing techniques such as
- Converting to lowercase
- Stop word Removal
- Removal of digits, special characters
- Stemming or Lemmatization but not both
- Count Vectorizer or TF-IDF Vectorizer but not both

Evaluate your model by:
- Providing input by yourself
- Creating a Confusion Matrix
- Calculating the Accuracy, Precision, Recall and F1-Score

In [220]:
import pandas as pd
import re
import nltk
import numpy as np

from nltk.corpus import stopwords
nltk.download("stopwords")
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('wordnet')

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, precision_score, recall_score, accuracy_score, balanced_accuracy_score, ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\carla\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\carla\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\carla\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [221]:
splits = {'train': 'unique_train_dataset.csv', 'validation': 'unique_validation_dataset.csv', 'test': 'unique_test_dataset.csv'}

**Training Set**

Use this to train your model

In [222]:
df_train = pd.read_csv("hf://datasets/mapsoriano/2016_2022_hate_speech_filipino/" + splits["train"])
df_train.head()

,text,label
0,Presidential candidate Mar Roxas implies that ...,1
1,Parang may mali na sumunod ang patalastas ng N...,1
2,Bet ko. Pula Ang Kulay Ng Posas,1
3,[USERNAME] kakampink,0
4,Bakit parang tahimik ang mga PINK about Doc Wi...,1


**Validation Set**

Use this set to evaluate your model

In [223]:
df_validation = pd.read_csv("hf://datasets/mapsoriano/2016_2022_hate_speech_filipino/" + splits["validation"])
df_validation.head()

,text,label
0,VinTee [USERNAME] [USERNAME] and [USERNAME] Ka...,1
1,binay's sidekicks were employees of makati cit...,1
2,This is expected as we use different methodol...,0
3,Ang tanga tanga talaga ni Nancy Binay eh. Tskkk.,1
4,Binay giving away bracelets after every selfie...,0


**Test Set**
  
Use this set to test your model

In [224]:
df_test = pd.read_csv("hf://datasets/mapsoriano/2016_2022_hate_speech_filipino/" + splits["test"])
df_test.head()

,text,label
0,Binay: Patuloy ang kahirapan dahil sa maling p...,0
1,SA GOBYERNONG TAPAT WELCOME SA BAGUO ANG LAHAT...,0
2,wait so ur telling me Let Leni Lead mo pero NY...,1
3,[USERNAME]wish this is just a nightmare that ...,0
4,doc willie ong and isko sabunutan po,0


## A. Understanding your training data

1. Check the first 10 rows of the training dataset

In [225]:
df_train.head(10)

,text,label
0,Presidential candidate Mar Roxas implies that ...,1
1,Parang may mali na sumunod ang patalastas ng N...,1
2,Bet ko. Pula Ang Kulay Ng Posas,1
3,[USERNAME] kakampink,0
4,Bakit parang tahimik ang mga PINK about Doc Wi...,1
5,"""Ang sinungaling sa umpisa ay sinungaling hang...",1
6,Leni Kiko,0
7,Nahiya si Binay sa Makati kaya dito na lang sa...,1
8,Another reminderHalalan,0
9,[USERNAME] Maybe because VP Leni Sen Kiko and ...,0


2. Check how many rows and columns are in the training dataset using `.info()`

In [226]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21773 entries, 0 to 21772
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    21773 non-null  object
 1   label   21773 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 340.3+ KB


3. Check for NaN values

In [227]:
df_train.isnull().sum()

text     0
label    0
dtype: int64

4. Check for duplicate rows

In [228]:
df_train.duplicated().sum()

0

5. Check how many rows belong to each class

In [229]:
df_train.count()

text     21773
label    21773
dtype: int64

## B. Text pre-processing

6. Remove duplicate rows

In [230]:
df_train.drop_duplicates()

,text,label
0,Presidential candidate Mar Roxas implies that ...,1
1,Parang may mali na sumunod ang patalastas ng N...,1
2,Bet ko. Pula Ang Kulay Ng Posas,1
3,[USERNAME] kakampink,0
4,Bakit parang tahimik ang mga PINK about Doc Wi...,1
...,...,...
21768,Marcos Talunan Marcos Magnanakaw,1
21769,Grabe kayo kay binay ??????????,0
21770,[USERNAME] Cnu ba naman ang hindImabibighani s...,0
21771,RT [USERNAME]: Tabi tabi yung mga nagsasabing ...,1


7. Remove rows with NaN values

In [231]:
df_train.dropna()

,text,label
0,Presidential candidate Mar Roxas implies that ...,1
1,Parang may mali na sumunod ang patalastas ng N...,1
2,Bet ko. Pula Ang Kulay Ng Posas,1
3,[USERNAME] kakampink,0
4,Bakit parang tahimik ang mga PINK about Doc Wi...,1
...,...,...
21768,Marcos Talunan Marcos Magnanakaw,1
21769,Grabe kayo kay binay ??????????,0
21770,[USERNAME] Cnu ba naman ang hindImabibighani s...,0
21771,RT [USERNAME]: Tabi tabi yung mga nagsasabing ...,1


8. Convert all text to lowercase

In [232]:
df_train['text'] = df_train['text'].str.lower()
df_train.head()

,text,label
0,presidential candidate mar roxas implies that ...,1
1,parang may mali na sumunod ang patalastas ng n...,1
2,bet ko. pula ang kulay ng posas,1
3,[username] kakampink,0
4,bakit parang tahimik ang mga pink about doc wi...,1


9. Remove digits, URLS and special characters

In [233]:
#remove puctuations/special characters/unneeded
df_train['text'] = df_train['text'].apply(lambda x: re.sub(r"[.,;:!\?\"'`]", "", x))
df_train['text'] = df_train['text'].apply(lambda x: re.sub(r"[@#$%^&*\/\+-_=\{\}<>]", "", x))
df_train['text'] = df_train['text'].apply(lambda x: re.sub(r"½m|½s|½t|½ï", "", x))

#remove links/email address
df_train['text'] = df_train['text'].apply(lambda x: re.sub(r"http\S+|www\.\S+", "", x))
df_train['text'] = df_train['text'].apply(lambda x: re.sub(r"\w+@\w+\.com", "", x))


10. Remove stop words

In [234]:
tagalog_stopwords = [
    "ako", "sa", "akin", "ko", "aking", "sarili", "kami", "atin", "ang", "aming",
    "amin", "ating", "ka", "iyong", "iyo", "inyong", "siya", "kanya", "mismo",
    "ito", "nito", "kanyang", "sila", "nila", "kanila", "kanilang", "kung", "ano",
    "alin", "sino", "kanino", "na", "mga", "iyon", "am", "ay", "maging", "naging",
    "mayroon", "may", "nagkaroon", "pagkakaroon", "gumawa", "ginagawa", "ginawa",
    "paggawa", "ibig", "dapat", "maaari", "marapat", "kong", "ikaw", "tayo", "hindi",
    "namin", "gusto", "nais", "niyang", "nilang", "niya", "huwag", "ginawang",
    "gagawin", "maaaring", "sabihin", "narito", "kapag", "ni", "nasaan", "bakit",
    "paano", "kailangan", "walang", "katiyakan", "isang", "at", "pero", "o", "dahil",
    "bilang", "hanggang", "habang", "ng", "pamamagitan", "para", "tungkol", "laban",
    "pagitan", "panahon", "bago", "pagkatapos", "itaas", "ibaba", "mula", "pataas",
    "pababa", "palabas", "ibabaw", "ilalim", "muli", "pa", "minsan", "dito", "doon",
    "saan", "lahat", "anumang", "kapwa", "bawat", "ilan", "karamihan", "iba", "tulad",
    "lamang", "pareho", "kaya", "kaysa", "masyado", "napaka", "isa", "bababa", "kulang",
    "marami", "ngayon", "kailanman", "sabi", "nabanggit", "din", "kumuha", "pumunta",
    "pumupunta", "ilagay", "makita", "nakita", "katulad", "mahusay", "likod", "kahit",
    "paraan", "noon", "gayunman", "dalawa", "tatlo", "apat", "lima", "una", "pangalawa",
    "si"
]


In [235]:
stop_words = tagalog_stopwords + stopwords.words("english")
df_train['text'] = df_train['text'].apply(lambda x: " ".join(word for word in x.split() if word not in stop_words))


In [236]:
df_train.head(10)

,text,label
0,presidential candidate mar roxas implies govt ...,1
1,parang mali sumunod patalastas nescaf coffee b...,1
2,bet pula kulay posas,1
3,username kakampink,0
4,parang tahimik pink doc willie ong reaction paper,1
5,sinungaling umpisa sinungaling dulo vp leni ma...,1
6,leni kiko,0
7,nahiya binay makati lang mandaluyong bwiset ha...,1
8,another reminderhalalan,0
9,username maybe vp leni sen kiko whole team buo...,0


11. Use Stemming or Lemmatization

In [237]:
wnl = WordNetLemmatizer()
df_train['text'] = df_train['text'].apply(lambda x: " ".join(wnl.lemmatize(word, "v") for word in x.split()))

df_train.head(10)

,text,label
0,presidential candidate mar roxas imply govt li...,1
1,parang mali sumunod patalastas nescaf coffee b...,1
2,bet pula kulay posas,1
3,username kakampink,0
4,parang tahimik pink doc willie ong reaction paper,1
5,sinungaling umpisa sinungaling dulo vp leni ma...,1
6,leni kiko,0
7,nahiya binay makati lang mandaluyong bwiset ha...,1
8,another reminderhalalan,0
9,username maybe vp leni sen kiko whole team buo...,0


In [238]:
tfidf_vectorizer = TfidfVectorizer()

tfidf_vectorizer.fit(df_train["text"])
tfidf_array = tfidf_vectorizer.transform(df_train["text"]).toarray()
data_cv = pd.DataFrame(tfidf_array, columns = tfidf_vectorizer.get_feature_names_out())

## C. Training your model

In [239]:
df_train['cleaned_text'] = df_train['text']
df_validation['cleaned_text'] = df_validation['text']
df_test['cleaned_text'] = df_test['text']

12. Put all text training data in variable **X_train**

In [240]:
X_train = tfidf_vectorizer.fit_transform(df_train['cleaned_text'])
X_validation = tfidf_vectorizer.transform(df_validation['cleaned_text'])
X_test = tfidf_vectorizer.transform(df_test['cleaned_text'])

13. Put all training data labels in variable **y_train**

In [241]:
y_train = df_train['label']
y_validation = df_validation['label']
y_test = df_test['label']

In [242]:
# X_train, X_test, Y_train, Y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

14. Use `CountVectorizer()` or `TfidfVectorizer()` to convert text data to its numerical form.

Put the converted data to **X_train_transformed** variable

15. Create an instance of `MultinomalNB()`

In [243]:
nb = MultinomialNB()

16. Train the model using `.fit()`

In [245]:
nb.fit(X_train, y_train)

MultinomialNB()

## D. Evaluate your model

17. Use `.predict()` to generate model predictions using the **validation dataset**


- Put all text validation data in **X_validation** variable

- Convert **X_validation** to its numerical form.

- Put the converted data to **X_validation_transformed**

- Put all predictions in **y_validation_pred** variable

In [246]:
y_pred_val = nb.predict(X_validation)


18. Get the Accuracy, Precision, Recall and F1-Score of the model using the **validation dataset**

- Put all validation data labels in **y_validation** variable

In [247]:
# y_validation = df_validation['label']

print("Validation Accuracy: ", accuracy_score(y_validation, y_pred_val))

Validation Accuracy:  0.8278571428571428


In [248]:

# put your answer here
print("Validation Accuracy: ", accuracy_score(y_validation, y_pred_val))
print(classification_report(y_validation, y_pred_val))

Validation Accuracy:  0.8278571428571428
              precision    recall  f1-score   support

           0       0.87      0.77      0.82      1385
           1       0.80      0.89      0.84      1415

    accuracy                           0.83      2800
   macro avg       0.83      0.83      0.83      2800
weighted avg       0.83      0.83      0.83      2800



19. Create a confusion matrix using the **validation dataset**

In [249]:
print("Confusion Matrix:")
print(confusion_matrix(y_validation, y_pred_val))

Confusion Matrix:
[[1063  322]
 [ 160 1255]]


In [250]:
print("Validation Accuracy: ", accuracy_score(y_validation, y_pred_val))

Validation Accuracy:  0.8278571428571428


20. Use `.predict()` to generate the model predictions using the **test dataset**


- Put all text validation data in **X_test** variable

- Convert **X_test** to its numerical form.

- Put the converted data to **X_test_transformed**

- Put all predictions in **y_test_pred** variable

In [251]:

# put your answer here
y_pred_test = nb.predict(X_test)

21. Get the Accuracy, Precision, Recall and F1-Score of the model using the **test dataset**

- Put all test data labels in **y_validation** variable



In [252]:
# y_test = df_test['label']

print("Test Accuracy: ", accuracy_score(y_test, y_pred_test))

Test Accuracy:  0.8288256227758007


In [253]:
# put your answer here
print("Validation Accuracy: ", accuracy_score(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test))

Validation Accuracy:  0.8288256227758007
              precision    recall  f1-score   support

           0       0.87      0.77      0.82      1412
           1       0.79      0.89      0.84      1398

    accuracy                           0.83      2810
   macro avg       0.83      0.83      0.83      2810
weighted avg       0.83      0.83      0.83      2810



22. Create a confusion matrix using the **test dataset**

In [254]:
# print("Confusion Matrix:")
# print(confusion_matrix(y_test, y_pred_test))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_test))

Confusion Matrix:
[[1090  322]
 [ 159 1239]]


## E. Test the model

23. Test the model by providing a non-hate speech input. The model should predict it as 0

In [260]:
# NEW: Testing Tagalog hate speech detection with new text input
new_text = pd.Series("Magandang umaga.")

# Transform the new text using the trained vectorizer (vect)
new_text_transform = tfidf_vectorizer.transform(new_text)

# Make the prediction using the trained Naive Bayes model (nb)
prediction = nb.predict(new_text_transform)
print(prediction)

# Interpret the prediction result
if prediction == 1:
    print("The sentence is classified as hate speech.")
else:
    print("The sentence is classified as non-hate speech.")

[0]
The sentence is classified as non-hate speech.


24. Test the model by providing a hate speech input. The model should predict it as 1

In [259]:
# put your answer here# NEW: Testing Tagalog hate speech detection with new text input
new_text = pd.Series("Ang sarap mong suntukin.")

# Transform the new text using the trained vectorizer (vect)
new_text_transform = tfidf_vectorizer.transform(new_text)

# Make the prediction using the trained Naive Bayes model (nb)
prediction = nb.predict(new_text_transform)
print(prediction)

# Interpret the prediction result
if prediction == 1:
    print("The sentence is classified as hate speech.")
else:
    print("The sentence is classified as non-hate speech.")

[1]
The sentence is classified as hate speech.
